In [ ]:
import os
import xml.etree.ElementTree as ET

def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = root.find('filename').text
    width = int(root.find('size').find('width').text)
    height = int(root.find('size').find('height').text)
    objects = []

    for obj in root.findall('object'):
        obj_name = obj.find('name').text
        xmin = int(obj.find('bndbox').find('xmin').text)
        ymin = int(obj.find('bndbox').find('ymin').text)
        xmax = int(obj.find('bndbox').find('xmax').text)
        ymax = int(obj.find('bndbox').find('ymax').text)
        objects.append((obj_name, (xmin, ymin), (xmax, ymax)))

    return filename, width, height, objects

def generate_annotation_v1(xml_folder, output_file):
    with open(output_file, 'w') as f:
        for xml_file in os.listdir(xml_folder):
            if xml_file.endswith('.xml'):
                xml_path = os.path.join(xml_folder, xml_file)
                filename, width, height, objects = parse_annotation(xml_path)
                
                f.write(f'# Compatible with PASCAL Annotation Version 1.00\n')
                f.write(f'Image filename : "{filename}"\n')
                f.write(f'Image size (X x Y x C) : {width} x {height} x 3\n')
                f.write(f'Database : "Unknown"\n')
                f.write(f'Objects with ground truth : {len(objects)} {{ ')
                for obj_name, _, _ in objects:
                    f.write(f'"{obj_name}" ')
                f.write('}\n')

                for i, (obj_name, (xmin, ymin), (xmax, ymax)) in enumerate(objects, start=1):
                    f.write(f'# Details for {obj_name} {i} ("{obj_name}")\n')
                    f.write(f'Original label for object {i} "{obj_name}" : "Unknown"\n')
                    f.write(f'Bounding box for object {i} "{obj_name}" (Xmin, Ymin) - (Xmax, Ymax) : ({xmin}, {ymin}) - ({xmax}, {ymax})\n')
                    f.write(f'Pixel mask for object {i} "{obj_name}" : "Unknown"\n\n')

                f.write('\n')

if __name__ == '__main__':
    xml_folder = '/path/to/xml/folder'
    output_file = '/path/to/output/annotations.txt'
    generate_annotation_v1(xml_folder, output_file)
